In [ ]:
import laser_lib
import numpy as np
import math
import time
#import cv2
#from skimage.color import rgb2lab, lab2rgb

In [ ]:
queue = laser_lib.DacQueue()

In [ ]:
# Circle
queue.dac_rate = 30000
T = 500
arr_pos = np.zeros((T, 2))
arr_col = np.zeros((T, 3))
arr_col[:, :] = 1
theta = np.linspace(0, 2*np.pi, T)
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

#arr_col = arr_col*np.expand_dims((theta > np.pi/2), 1)

while(True):
    queue.submit(arr_pos, arr_col, loop=False)
queue.submit(arr_pos, np.zeros((T,3)), loop=True)

In [ ]:
import numpy as np

# Settings
queue.dac_rate = 30000
points_per_unit = 500
r_outer, r_inner = 0.4, 0.15 
transition_points = 20
laser_offset = -1
overshoot = 0.0275
offset_distance = 0.5
pupil_height = r_outer * 0.45

# 1. Generate Local Geometry (One nested pair centered at 0,0)
t_out = int(points_per_unit * r_outer)
t_in = int(points_per_unit * r_inner)

theta_out = np.linspace(-overshoot * 2*np.pi, 2*np.pi + 2*np.pi * overshoot, t_out)
pts_out = np.column_stack([r_outer * np.cos(theta_out), r_outer * np.sin(theta_out), np.zeros(t_out)])

theta_in = np.linspace(-overshoot * 2*np.pi, 2*np.pi + 2*np.pi * overshoot, t_in)
pts_in = np.column_stack([r_inner * np.cos(theta_in), r_inner * np.sin(theta_in), pupil_height * np.ones(t_in)])

# Path from end of outer to start of inner
trans_out_to_in = np.linspace(pts_out[-1], pts_in[0], transition_points)

# Create a single "Nested Pair" template
local_pair = np.vstack([pts_out, trans_out_to_in, pts_in])
num_pts_pair = len(local_pair)

# 2. Rotation Loop
angle_x, angle_y = 0.0, 0.0
speed = 0.01
max_angle = 2*np.pi/6

while True:
    # Define Rotation Matrices
    cx, sx = np.cos(angle_x), np.sin(angle_x)
    cy, sy = np.cos(angle_y), np.sin(angle_y)
    rx = np.array([[1, 0, 0], [0, cx, -sx], [0, sx, cx]])
    ry = np.array([[cy, 0, sy], [0, 1, 0], [-sy, 0, cy]])
    R = (ry @ rx).T

    # 3. ROTATE LOCALLY first
    rotated_pair = local_pair @ R

    # 4. TRANSLATE to independent positions
    pair_left = rotated_pair.copy()
    pair_left[:, 0] -= offset_distance
    
    pair_right = rotated_pair.copy()
    pair_right[:, 0] += offset_distance

    # 5. Connect the two rotated pairs with blanking transitions
    trans_l_to_r = np.linspace(pair_left[-1], pair_right[0], transition_points)
    trans_r_to_l = np.linspace(pair_right[-1], pair_left[0], transition_points)
    
    full_frame_3d = np.vstack([pair_left, trans_l_to_r, pair_right, trans_r_to_l])
    
    # 6. Setup Colors (Dynamic size based on frame assembly)
    arr_col = np.zeros((len(full_frame_3d), 3))
    
    # Color Left Pair
    arr_col[3 : t_out-3, :] = 1 # Left Outer
    arr_col[t_out+transition_points+3 : num_pts_pair-3, :] = 1 # Left Inner
    
    # Color Right Pair
    start_right = num_pts_pair + transition_points
    arr_col[start_right+3 : start_right+t_out-3, :] = 1 # Right Outer
    arr_col[start_right+t_out+transition_points+3 : start_right+num_pts_pair-3, :] = 1 # Right Inner

    arr_col = np.roll(arr_col, -laser_offset, axis=0)
    
    # Submit 2D projection
    queue.submit(full_frame_3d[:, :2], arr_col, loop=False)
    
    angle_x += speed
    angle_y += speed * 0.6
    #if(abs(angle_y) > max_angle):
    #    speed *= -1